# 🤖 Hybrid Multi-Agent System Showcase

**Author:** Juan Jose Pardo
**Project:** A comprehensive demonstration for Wholesale proyect Premium Choice using AI reference technologies: Multi-Agent Systems, Langchain, LlamaIndex, RAG, and Agent Frameworks (MCP).

### Objective
This notebook builds a sophisticated multi-agent system that handles both **structured data** (Excel sales files) and **unstructured data** (text-based incident reports).

This architecture directly demonstrates the required skills:
- **Langchain:** Used for the main orchestrator agent and the structured data tool.
- **LlamaIndex:** Used to build a classic RAG pipeline for unstructured Q&A.
- **RAG (Retrieval-Augmented Generation):** Implemented via LlamaIndex for querying text documents.
- **Multi-Agent System & MCP (Multi-Agent Coordination Protocol):** The core design features a primary agent that routes tasks to specialized sub-agents, acting as a coordination protocol.

## 1. Setup and Dependencies

Install all necessary libraries from Langchain, LlamaIndex, and OpenAI.

In [1]:
!pip install langchain langchain-openai langchain-experimental openai pandas openpyxl tabulate llama-index python-dotenv


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
#Import API Key associated with LLM (Large Language Model) ChatGPT from OpenAI
import os
import pandas as pd
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain import hub # For pulling react prompts
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

# Load environment variables from .env file
load_dotenv()

# Verify that the key has been loaded correctly
if 'OPENAI_API_KEY' not in os.environ:
    print("Error: The OPENAI_API_KEY environment variable was not found.")
    print("Please create a .env file and add your key.")
else:
    print("API Key Loaded.")

# Configure LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


API Key Loaded.


## 3. Part 1: The Structured Data Agent (with Langchain)

First, we create a specialized tool for handling the structured data from the Excel files. This tool is a self-contained Pandas DataFrame Agent.

In [3]:
# --- Code for Part 1: Creation of the Structured Data Agent (Corrected) ---

# Import necessary libraries
import glob
from langchain.agents import Tool
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

def load_and_prepare_data(base_path='../data/raw/'):
    """
    Loads all .xlsx files from the specified directory.
    It identifies files that follow a temporal pattern (e.g., 'sales_2023.xlsx') and concatenates them.
    Returns a list of pandas DataFrames.
    """
    if not os.path.isdir(base_path):
        print(f"Error: The directory '{base_path}' was not found. Please verify the project structure.")
        return []
        
    all_files = glob.glob(os.path.join(base_path, "*.xlsx"))
    
    dataframes = {}
    prefixes = ['compras_dcto_items', 'compras_dcto', 'ventas_dcto_items', 'ventas_dcto']
    single_files = ['articulos.xlsx', 'clientes.xlsx', 'proveedores.xlsx']
    
    for f_name in single_files:
        full_path = os.path.join(base_path, f_name)
        if os.path.exists(full_path):
            df_name = f_name.split('.')[0]
            dataframes[df_name] = pd.read_excel(full_path)
            print(f"Loaded: {f_name} as '{df_name}'")
        else:
             print(f"Warning: File not found {full_path}")

    for prefix in prefixes:
        files_to_concat = [f for f in all_files if os.path.basename(f).startswith(prefix)]
        if not files_to_concat: continue
            
        df_list = [pd.read_excel(f) for f in files_to_concat]
        concatenated_df = pd.concat(df_list, ignore_index=True)
        dataframes[prefix] = concatenated_df
        print(f"Loaded and concatenated {len(df_list)} files for prefix '{prefix}'")
    
    print("\nAll structured data has been loaded into memory.")
    return list(dataframes.values())

# Define the relative path to the data directory.
data_path = '../data/raw/'
structured_dfs = load_and_prepare_data(base_path=data_path)

# The Pandas Agent is created only if the dataframes were loaded successfully.
if structured_dfs:
    pandas_agent = create_pandas_dataframe_agent(
        llm,
        structured_dfs,
        verbose=True,
        agent_executor_kwargs={"handle_parsing_errors": True},
        allow_dangerous_code=True  # <-- ESTA ES LA LÍNEA QUE AÑADIMOS
    )

    # The agent is then wrapped in a Langchain 'Tool' for use by the main orchestrator.
    structured_data_tool = Tool(
        name="Structured Data Analyst",
        func=pandas_agent.invoke,
        description="""
        Use this tool for any questions about structured business data.
        It can answer questions about sales, purchases, customers, articles, and suppliers
        by querying a set of pandas DataFrames.
        Input should be a full question about business operations.
        """
    )
    print("\nStructured Data Agent Tool created successfully!")
else:
    print("\nCould not create the structured data tool because no data was loaded.")

Loaded: articulos.xlsx as 'articulos'
Loaded: clientes.xlsx as 'clientes'
Loaded: proveedores.xlsx as 'proveedores'
Loaded and concatenated 7 files for prefix 'compras_dcto_items'
Loaded and concatenated 14 files for prefix 'compras_dcto'
Loaded and concatenated 15 files for prefix 'ventas_dcto_items'
Loaded and concatenated 28 files for prefix 'ventas_dcto'

All structured data has been loaded into memory.

Structured Data Agent Tool created successfully!


### 4. Part 2: The Unstructured Data Agent (RAG with LlamaIndex)

With the structured data tool complete, the next step is to build a tool that can query unstructured knowledge. This is a classic Retrieval-Augmented Generation (RAG) pipeline. A sample business report is created in a text file. **LlamaIndex** is then used to build a vector index from this document, creating a query engine that can answer questions based on its content.

In [4]:
# --- Code for Part 2: Creation of the RAG Agent Tool ---

# Import necessary libraries from LlamaIndex
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.llms.openai import OpenAI as LlamaOpenAI
from llama_index.core.tools import QueryEngineTool


# A directory is created to hold unstructured data sources.
os.makedirs("unstructured_data", exist_ok=True)

# A sample business report is created programmatically.
# This report provides context for the 'why' behind the data, based on the EDA findings.
report_text = """
Business Strategy Report - 2022 Review

The significant growth experienced starting in mid-2022 is directly attributable to two key initiatives. 
First, in May 2022, Premium Choice secured an exclusive distribution agreement with 'ST DALF', a popular French gourmet food brand. 
This allowed us to corner the market on their highly sought-after jams.

Second, in June 2022, we launched a targeted B2B sales campaign aimed at major supermarket chains, 
offering them preferential pricing. This strategy successfully brought on board key accounts, including Cencosud and Almacenes Éxito, 
which became major purchasers of the new product lines.
"""

with open("unstructured_data/business_report_2022.txt", "w") as f:
    f.write(report_text)

print("Unstructured report file created.")

# LlamaIndex's SimpleDirectoryReader loads the text file(s).
documents = SimpleDirectoryReader("./unstructured_data").load_data()
print(f"Loaded {len(documents)} unstructured document(s).")

# The LLM for LlamaIndex is configured separately.
Settings.llm = LlamaOpenAI(model="gpt-3.5-turbo", temperature=0)

# A vector store index is created from the documents.
index = VectorStoreIndex.from_documents(documents)

# The query engine is the core of the RAG pipeline.
query_engine = index.as_query_engine()
print("LlamaIndex RAG query engine created.")

# The LlamaIndex query engine is wrapped in a Langchain-compatible Tool.
unstructured_data_tool = Tool(
    name="Unstructured Knowledge Base",
    func=lambda q: str(query_engine.query(q)), # The query engine's response must be converted to a string.
    description="""
    Use this tool for any questions about unstructured knowledge, business strategy,
    marketing reports, or 'why' questions that are not in the structured data.
    Input should be a specific question about information that might be in text documents.
    """
)

print("\nUnstructured Data RAG Tool created successfully!")

Unstructured report file created.
Loaded 1 unstructured document(s).
LlamaIndex RAG query engine created.

Unstructured Data RAG Tool created successfully!


### 5. Part 3: The Orchestrator Agent (The MCP)

With both specialized tools ready, the final step is to create the main agent. This "Orchestrator" or "meta-agent" does not answer questions itself. Its sole purpose is to understand the user's query and route it to the correct tool. This routing logic functions as the **Multi-Agent Coordination Protocol (MCP)** for this system. It uses a ReAct (Reasoning and Acting) prompt to decide which tool to use, and in what sequence.

In [5]:
# --- Code for Part 3: Assembling the Orchestrator Agent ---

# The list of tools that the main agent can choose from is defined.
tools = [structured_data_tool, unstructured_data_tool]

# A pre-built ReAct (Reasoning and Acting) prompt template is pulled from Langchain Hub.
# This prompt guides the LLM on how to reason about which tool to use.
prompt = hub.pull("hwchase17/react")

# The main orchestrator agent is created using the LLM, the list of tools, and the prompt.
orchestrator_agent = create_react_agent(llm, tools, prompt)

# The AgentExecutor is the runtime for the agent. It's what actually runs the agent's logic.
agent_executor = AgentExecutor(
    agent=orchestrator_agent,
    tools=tools,
    verbose=True, # Set to True to see the agent's "thought process".
    handle_parsing_errors=True
)

print("Orchestrator Agent and Executor created successfully. The system is ready.")

Orchestrator Agent and Executor created successfully. The system is ready.


/Users/juanpardo/.pyenv/versions/3.10.14/lib/python3.10/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


### 6. Final Demonstration: The Hybrid System in Action

With the entire system assembled, it's time for the final validation. The following tests will demonstrate the orchestrator agent's ability to correctly route queries to the appropriate specialized tool. The `verbose=True` setting allows us to see the agent's "chain of thought" as it reasons about which tool to use.

In [6]:
# --- Query 1: Should be routed to the Structured Data Analyst ---
print("\n--- DEMO 1: Asking a question about structured data (sales figures) ---")

# Based on our EDA, we know 2023 was a high-growth year.
q1 = "What were the total sales (subtotal) for the year 2023?"

agent_executor.invoke({"input": q1})


--- DEMO 1: Asking a question about structured data (sales figures) ---


> Entering new AgentExecutor chain...
I should use the Structured Data Analyst tool to find the total sales for the year 2023.
Action: Structured Data Analyst
Action Input: {"question": "What were the total sales (subtotal) for the year 2023?"}

> Entering new AgentExecutor chain...
Thought: To calculate the total sales (subtotal) for the year 2023, we need to sum the 'subtotal' column from all dataframes where the 'defecha' column is in the year 2023.

Action: [python_repl_ast]
Action Input: df1['subtotal'].sum() + df2['subtotal'].sum() + df3['subtotal'].sum() + df4['subtotal'].sum() + df5['subtotal'].sum() + df6['subtotal'].sum() + df7['subtotal'].sum()[python_repl_ast] is not a valid tool, try one of [python_repl_ast].I should use the [python_repl_ast] tool to calculate the total sales for the year 2023 by summing the 'subtotal' column from all dataframes where the 'defecha' column is in the year 2023.
Action

{'input': 'What were the total sales (subtotal) for the year 2023?',
 'output': 'The total sales for the year 2023 are not available through the tools provided.'}